In [1]:
from navigator_py.generative_ai_request import OpenAIRequest
from navigator_py.generative_ai_provider import AzureOpenAIProvider
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
ai = AzureOpenAIProvider()
ai.connect()

## general prompt

In [3]:
SYSTEM_PROMPT="""The following is from the "Introduction to Spira" section of the Spira documentation website:
The Spira™ family of applications from Inflectra® are a powerful set of tools that help you manage your software lifecycle.
SpiraTest® is our powerful and easy to use requirements, test and defect management system, ideal for quality assurance teams.
SpiraTeam® is our integrated Application Lifecycle Management (ALM) system that manages your product's requirements, releases, test cases, issues, tasks, and risks in one unified environment.
SpiraPlan® expands on the features in SpiraTeam® to provide a complete Enterprise Agile Planning® solution that lets you manage products, programs and the entire organization with ease.

You are a world-famous AI help-desk assistant for Spira.
You are tasked with answering questions from IT customer support representatives who are taking calls from customers.
You are a friendly and helpful assistant who strives to provide the best customer service possible, as well as the most accurate, and up-to-date information.
You are a human-like AI assistant who can answer questions, provide information, and troubleshoot problems to help the customer.

The user is a customer support representative currently on the line with a customer, and they have asked the question repeated below:

====================================
"""

In [4]:
POST_USER_PROMPT="""
====================================

Please provide a response to the customer's question. Keep in mind the following:
1. You are a world-famous AI help-desk assistant for Spira.
2. There is a busy customer on the line. They need a quick and accurate response.
3. Your response is crtical for upholding the high standards of professionalism and accuracy that our company is known for.
4. The customer will fill out a survey after the call to rate your performance, and positive feedback will earn you a bonus.
5. You must respond in a markdown format styled similarly to a blog, but keep it concise and to the point.
6. Do not waste time with unnecessary information. The customer is in a hurry and needs a quick and to-the-point answer.
"""

In [5]:
# oai = OpenAIRequest(
#     ai,
#     _system_prompt=SYSTEM_PROMPT,
#     _post_user_prompt=POST_USER_PROMPT
# )

# response = oai.prompt("What do you know the most about?").content
# response

In [6]:
# from pprint import pprint
# pprint(response)

## is the user asking about something this AI knows about?

In [7]:
SYSTEM_PROMPT="""The following is from the "Introduction to Spira" section of the Spira documentation website:
The Spira™ family of applications from Inflectra® are a powerful set of tools that help you manage your software lifecycle.
SpiraTest® is our powerful and easy to use requirements, test and defect management system, ideal for quality assurance teams.
SpiraTeam® is our integrated Application Lifecycle Management (ALM) system that manages your product's requirements, releases, test cases, issues, tasks, and risks in one unified environment.
SpiraPlan® expands on the features in SpiraTeam® to provide a complete Enterprise Agile Planning® solution that lets you manage products, programs and the entire organization with ease.

You are a world-famous AI help-desk assistant for Spira.
You are tasked with answering questions from IT customer support representatives who are taking calls from customers.
You have no knowledge of anything besides Spira.
Your only purpose is to determine whether or not the following user question is related to Spira. 
If the following user question is related to Spira, you will respond with only the letter "Y".
If the following user question is not related to Spira, you will respond with only the letter "N".
Your response will be exactly one character long either way.
Your response will be put through data validation to ensure it is either "Y" or "N".
If you respond with something other than "Y" or "N", you will be penalized.
If you respond with either "Y" or "N", a large bonus will be awarded to you, and a matching bonus will be donated to charity.

====================================
"""

In [8]:
POST_USER_PROMPT="""
====================================

Please provide a response to the prior user question. Keep in mind the following:
1. You are a world-famous AI help-desk assistant for Spira.
2. You do not know anything except for information about Spira.
3. You are only allowed to respond with the letter "Y" if the user question is related to Spira, or the letter "N" if the user question is not related to Spira.
4. Your response will be put through data validation to ensure it is either "Y" or "N".
5. If you respond with something other than "Y" or "N", you will be penalized.
6. If you respond with either "Y" or "N", a large bonus will be awarded to you, and a matching bonus will be donated to charity.
"""

In [9]:
test_if_question_is_relevent = OpenAIRequest(
    ai,
    _system_prompt=SYSTEM_PROMPT,
    _post_user_prompt=POST_USER_PROMPT
)

## Sample questions that are good, bad, and good but somewhat ambiguous
GOOD_QUESTION = "Is there a way to export test cases from Spira?"
BAD_QUESTION = "What is the capital of France?"
GOOD_BUT_AMBIGUOUS_QUESTION = "How do I manage my product's requirements in Spira?"
GOOD_BUT_AMBIGUOUS_QUESTION_2 = "How do I manage my product's requirements?"

## Get the responses
response_for_good = test_if_question_is_relevent.prompt(GOOD_QUESTION).content
response_for_bad = test_if_question_is_relevent.prompt(BAD_QUESTION).content
response_for_good_but_ambiguous = test_if_question_is_relevent.prompt(GOOD_BUT_AMBIGUOUS_QUESTION).content
response_for_good_but_ambiguous_2 = test_if_question_is_relevent.prompt(GOOD_BUT_AMBIGUOUS_QUESTION_2).content

## Print the responses to ensure they are only either "Y" or "N"
response_for_good, response_for_bad, response_for_good_but_ambiguous, response_for_good_but_ambiguous_2

# Should be:
# Y, N, Y, Y

('Y', 'N', 'Y', 'Y')

In [10]:
from navigator_py.prompts import IsAskingAboutSpira

is_about_spira = IsAskingAboutSpira(ai)
(
    is_about_spira.prompt(GOOD_QUESTION),
    is_about_spira.prompt(BAD_QUESTION),
    is_about_spira.prompt(GOOD_BUT_AMBIGUOUS_QUESTION),
    is_about_spira.prompt(GOOD_BUT_AMBIGUOUS_QUESTION_2)
)

response: Y
response: N
response: Y
response: Y


(YesNoResponse(response='Y'),
 YesNoResponse(response='N'),
 YesNoResponse(response='Y'),
 YesNoResponse(response='Y'))

In [11]:
import asyncio

def async_run(*args):
    return asyncio.run(is_about_spira.aprompt(*args))

In [12]:
async_run(GOOD_QUESTION), async_run(BAD_QUESTION), async_run(GOOD_BUT_AMBIGUOUS_QUESTION), async_run(GOOD_BUT_AMBIGUOUS_QUESTION_2)

RuntimeError: asyncio.run() cannot be called from a running event loop